This script lets user to calculate mAP for rectangular bounding boxes

In [1]:
# Loading all lib
import pandas as pd
import glob
import cv2
import numpy as np
import math
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Rectangle
from random import shuffle

Functions needed to load and process data. They are described in script about final model for rectangular bboxes.

In [2]:
##Usefull functions

def create_df(path_annotations_circle):
    all_files = glob.glob(path_annotations_circle + "\\*.csv")
    df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    df_circle = df[['name', 'c-x', 'c-y', 'radius']]

    return df_circle

def get_max_number(df_circle):
    value = df_circle.groupby('name').agg('count').sort_values('radius', ascending=False).max()[['radius']].tolist()
    return value[0]

def get_bbox_cords(df_circle):
    gb = df_circle.groupby('name')    
    gb = [gb.get_group(x) for x in gb.groups]


    list =[]
    for set_of_df in gb:
        lista = []
        for i in range(18*3):
            if str(set_of_df['radius'].values.tolist()[0]) == 'nan':
                lista.append(0)
            else:
                lista = set_of_df[['c-x','c-y','radius']].stack().tolist()
                number = 54 - len(lista)
                a = [0]*number
                lista = lista + a
                break
        list.append(lista)

    return list

def get_bbox_cords_to_dict(df_circle):
    gb = df_circle.groupby('name')    
    gb = [gb.get_group(x) for x in gb.groups]

    dict = {}
    list =[]
    for set_of_df in gb:
        name = (set_of_df['name'].values.tolist()[0].replace('.csv', ''))
        lista = []
        for i in range(18*3):
            if str(set_of_df['radius'].values.tolist()[0]) == 'nan':
                lista.append(0)
            else:
                lista = set_of_df[['c-x','c-y','radius']].stack().tolist()
                number = 54 - len(lista)
                a = [0]*number
                lista = lista + a
                break
        list.append(lista)
        dict.update({name:lista})



    return dict

def get_image_data(imagespath):
  path = glob.glob(imagespath + '\\*.png')
  images=[cv2.imread(file) for file in path]
  return images


def apply_threshold_and_split_pred(trainprediction, thresh):
    prediction = []
    three_coord_list = []
    iter = 0
    prediction_threshold = trainprediction

    for i in range(len(prediction_threshold[0])):
        if prediction_threshold[0][i] < thresh:
            prediction_threshold[0][i] = 0

    for elem in prediction_threshold[0]:
        if iter < 4:
            three_coord_list.append(elem)
            iter +=1
        else:
            iter = 1
            prediction.append(three_coord_list)
            three_coord_list = []
            three_coord_list.append(elem)
    prediction.append(three_coord_list)

    return prediction

def split_origin(origin_img):
    origin = []
    three_coord_list = []
    iter = 0

    for elem in origin_img:
        if iter < 4:
            three_coord_list.append(elem)
            iter +=1
        else:
            iter = 1
            origin.append(three_coord_list)
            three_coord_list = []
            three_coord_list.append(elem)
    origin.append(three_coord_list)

    return origin

def draw_circle(ax, Color, coords):
    for elem in coords:
        w = float(elem[2])
        h = float(elem[3])        
        x = float(elem[0]) - w/2
        y = float(elem[1]) - h/2
        circ = Rectangle((x,y),w,h, fill=False, color = Color)
        ax.add_patch(circ)
    return ax


def plot_results(img, prediction, original_coords):

    fig,ax = plt.subplots(1)
    ax.imshow(img, interpolation='nearest')

    ax = draw_circle(ax, 'Red', prediction)
    ax = draw_circle(ax, 'Blue', original_coords)

    plt.show()

def calculate_iou(prediction, original_coords):
    iter = 0 
    iou_list = []
    for pred_elem, original_elem in zip(prediction,original_coords):
        iter +=1
        c1_x = pred_elem[0]
        c1_y = pred_elem[1] 
        r1 = pred_elem[2] 
        c2_x = original_elem[0]
        c2_y = original_elem[1]
        r2 = original_elem[2]

        if c2_x == 0 and c2_y == 0 and r2 == 0:
            iou = 'NoOriginObject'
        else:
            distance = np.sqrt((c1_x - c2_x)**2 + (c1_y - c2_y)**2)
            
            if distance > r1 + r2:
                return 0.0
            elif (distance <= (r1 - r2) and r1 >= r2):
                intersection_area = np.pi * r2 * r2
        
            elif (distance <= (r2 - r1) and r2 >= r1):
                intersection_area = np.pi * r1 * r1
        
            else :
                alpha = np.arccos(((r1 * r1) + (distance * distance) - (r2 * r2)) / (2 * r1 * distance)) * 2
                beta = np.arccos(((r2 * r2) + (distance * distance) - (r1 * r1)) / (2 * r2 * distance)) * 2
                
                a1 = (0.5 * beta * r2 * r2 ) - (0.5 * r2 * r2 * np.sin(beta))
                a2 = (0.5 * alpha * r1 * r1) - (0.5 * r1 * r1 * np.sin(alpha))
                intersection_area = a1 + a2
                
            area1 = np.pi * r1**2
            area2 = np.pi * r2**2

            union_area = area1 + area2 - intersection_area
                
            iou = intersection_area / union_area

        iou_list.append(iou)
        print(f'{iter}: Original: {original_elem} | Predicted: {pred_elem} | IOU: {iou}')

    return iou_list

def calculate_iou_with_probability(prediction, original_coords, probability):
    iter = 0 
    i = 0
    iou_list = []
    for pred_elem, original_elem in zip(prediction,original_coords):
        iter +=1
        c1_x = pred_elem[0]
        c1_y = pred_elem[1] 
        r1 = pred_elem[2] 
        c2_x = original_elem[0]
        c2_y = original_elem[1]
        r2 = original_elem[2]

        if c2_x == 0 and c2_y == 0 and r2 == 0:
            iou = 'NoOriginObject'
        else:
            distance = np.sqrt((c1_x - c2_x)**2 + (c1_y - c2_y)**2)
            
            if distance > r1 + r2:
                return 0.0
            elif (distance <= (r1 - r2) and r1 >= r2):
                intersection_area = np.pi * r2 * r2
        
            elif (distance <= (r2 - r1) and r2 >= r1):
                intersection_area = np.pi * r1 * r1
        
            else :
                alpha = np.arccos(((r1 * r1) + (distance * distance) - (r2 * r2)) / (2 * r1 * distance)) * 2
                beta = np.arccos(((r2 * r2) + (distance * distance) - (r1 * r1)) / (2 * r2 * distance)) * 2
                
                a1 = (0.5 * beta * r2 * r2 ) - (0.5 * r2 * r2 * np.sin(beta))
                a2 = (0.5 * alpha * r1 * r1) - (0.5 * r1 * r1 * np.sin(alpha))
                intersection_area = a1 + a2
                
            area1 = np.pi * r1**2
            area2 = np.pi * r2**2

            union_area = area1 + area2 - intersection_area
                
            iou = intersection_area / union_area

        iou_list.append(iou)
        print(f'{iter}: Original: {original_elem} | Predicted: {pred_elem} | Probability: {round(float(probability[0][iter-1]), 3)} | IOU: {iou}')
        i = i + 1

    return iou_list

def calculate_prob(list_of_coords):
    probability = []
    list_of_prob = []
    for set_ in list_of_coords:
        lst = set_[::3]
        for value in lst:
            if value != 0:
                probability.append(1)
            else:
                probability.append(0)
        list_of_prob.append(probability)
        probability = []
    
    return list_of_prob


def dataset_shuffle(images,list_of_prob,list_of_coords,bool):
    if bool == True:
        whole_set = []
        for img, prob, coord in zip(images,list_of_prob ,list_of_coords):
            one_set = []
            one_set.append(img)
            one_set.append(prob)
            one_set.append(coord)
            whole_set.append(one_set)

        shuffle(whole_set)
        img_sh = []
        prob_sh =[]
        coord_sh = []

        for set_ in whole_set:
            img_sh.append(set_[0])
            prob_sh.append(set_[1])
            coord_sh.append(set_[2])

        return img_sh, prob_sh, coord_sh
    else:
        return images, list_of_prob, list_of_coords

def reverse_transform_data_only_coords(data, grid_size):
    image_height, image_width = 202, 308
    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]
    
    coords_final = []

    for batch_data in data.reshape((-1, grid_size[0] * grid_size[1], 3)):
        batch_coords = []

        for y in range(grid_size[0]):
            for x in range(grid_size[1]):
                local_x, local_y, r = batch_data[y * grid_size[1] + x] 
                global_x = x * cell_width + local_x
                global_y = y * cell_height + local_y
                
                batch_coords.extend([global_x, global_y, r])

        coords_final.append(batch_coords)

    return coords_final

def reverse_transform_deep_nested_coords(batch_data, grid_size, max_box_per_apple=2):
    image_height, image_width = 202, 308
    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]
    
    output_list = []

    for y in range(grid_size[0]):
        for x in range(grid_size[1]):

            if max_box_per_apple == 2:
                local_coords1 = batch_data[0][y][x][0]
                local_coords2 = batch_data[0][y][x][1]
                
                global_x1 = x * cell_width + local_coords1[0]
                global_y1 = y * cell_height + local_coords1[1]
                
                global_x2 = x * cell_width + local_coords2[0]
                global_y2 = y * cell_height + local_coords2[1]
                
                output_list.append(global_x1)
                output_list.append(global_y1)
                output_list.append(local_coords1[2])
                output_list.append(global_x2)
                output_list.append(global_y2)
                output_list.append(local_coords2[2])
            elif max_box_per_apple == 1:
                local_coords1 = batch_data[0][y][x][0]
                
                global_x1 = x * cell_width + local_coords1[0]
                global_y1 = y * cell_height + local_coords1[1]
                
                output_list.append(global_x1)
                output_list.append(global_y1)
                output_list.append(local_coords1[2])


    return np.array(output_list)
    
def reverse_transform_deep_nested_coords_and_normalization(batch_data, grid_size, max_box_per_apple=2):
    image_height, image_width = 202, 308
    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]
    
    output_list = []

    for y in range(grid_size[0]):
        for x in range(grid_size[1]):

            if max_box_per_apple == 2:
                local_coords1 = batch_data[0][y][x][0]
                local_coords2 = batch_data[0][y][x][1]
                
                global_x1 = x * cell_width + local_coords1[0] * cell_width
                global_y1 = y * cell_height + local_coords1[1] * cell_height
                
                global_x2 = x * cell_width + local_coords2[0] * cell_width
                global_y2 = y * cell_height + local_coords2[1] * cell_height
                
                output_list.append(global_x1)
                output_list.append(global_y1)
                output_list.append(local_coords1[2] * cell_width)
                output_list.append(local_coords1[3] * cell_height)
                output_list.append(global_x2)
                output_list.append(global_y2)
                output_list.append(local_coords2[2] * cell_width)
                output_list.append(local_coords2[2] * cell_height)
            elif max_box_per_apple == 1:
                local_coords1 = batch_data[0][y][x][0]
                
                global_x1 = x * cell_width + local_coords1[0] * cell_width
                global_y1 = y * cell_height + local_coords1[1] * cell_height
                
                output_list.append(global_x1)
                output_list.append(global_y1)
                output_list.append(local_coords1[2] * cell_width)
                output_list.append(local_coords1[3] * cell_height)


    return np.array(output_list)

def Iou(prediction, original_coords):
    iter = 0 
    iou_list = []
    for pred_elem, original_elem in zip(prediction, original_coords):
        iter += 1
        x1_pred, y1_pred, width1_pred, height1_pred = pred_elem
        x1_pred = x1_pred - width1_pred/2
        y1_pred = y1_pred - height1_pred/2
        x2_pred = x1_pred + width1_pred
        y2_pred = y1_pred + height1_pred
        
        x1_orig, y1_orig, width1_orig, height1_orig = original_elem
        x1_orig = x1_orig - width1_orig/2
        y1_orig = y1_orig - height1_orig/2
        x2_orig = x1_orig + width1_orig
        y2_orig = y1_orig + height1_orig

        x1_intersection = max(x1_pred, x1_orig)
        y1_intersection = max(y1_pred, y1_orig)
        x2_intersection = min(x2_pred, x2_orig)
        y2_intersection = min(y2_pred, y2_orig)

        if x1_orig ==0 and y1_orig == 0 and width1_orig == 0 and height1_orig == 0:
            return [-1]

        if x1_intersection < x2_intersection and y1_intersection < y2_intersection:
            intersection_area = (x2_intersection - x1_intersection) * (y2_intersection - y1_intersection)
        else:
            intersection_area = 0.0

        area1_pred = width1_pred * height1_pred
        area1_orig = width1_orig * height1_orig

        union_area = area1_pred + area1_orig - intersection_area

        if union_area == 0:
            iou = 0.0
        else:
            iou = intersection_area / union_area

        iou_list.append(iou)

    return iou_list

def NMS(pred, iou_threshold, prob_threshold, prob):

    one_box = []
    predictions_with_prob =[]
    iter = 0
    iter_prob = 0
    for elem in pred:
        if iter == 3:
            iter = 0
            one_box.append(elem)
            one_box.append(prob[iter_prob])
            iter_prob += 1
            predictions_with_prob.append(one_box)
            one_box = []
        else:
            one_box.append(elem)
            iter += 1

    bbox_list_thresholded = []
    bbox_list_new = []

    box_sorted = sorted(predictions_with_prob, reverse=True, key = lambda x :x[4])

    for box in box_sorted:
        if box[4] > prob_threshold:
            bbox_list_thresholded.append(box)

    while len(bbox_list_thresholded) > 0:
        current_box = bbox_list_thresholded.pop(0)
        bbox_list_new.append(current_box)
        for box in bbox_list_thresholded:
            iou = Iou([current_box[:4]], [box[:4]])
            if iou[0] > iou_threshold:
                bbox_list_thresholded.remove(box)

    return bbox_list_new

def transform_data(coords, probs, grid_size, max_apples_per_cell=2):
    image_height, image_width = 202, 308
    
    transformed_coords_final = []
    transformed_probs_final = []

    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]

    for i in range(len(probs)):
        transformed_coords = np.zeros((grid_size[0], grid_size[1], max_apples_per_cell, 3))
        transformed_probs = np.zeros((grid_size[0], grid_size[1], max_apples_per_cell, 1))

        apple_counter = np.zeros((grid_size[0], grid_size[1]), dtype=int)

        for j in range(len(probs[0])):
            if probs[i][j] == 1:  
                x, y, r = coords[i][j*3:j*3+3]

                if x == image_width and y == image_height:
                    cell_x = 3
                    cell_y = 3
                elif x == image_width:
                    cell_x = 3
                    cell_y = int(np.floor(y // cell_height))
                elif y == image_height:
                    cell_x = int(np.floor(x // cell_width))
                    cell_y = 3
                else:
                    cell_x, cell_y = int(np.floor(x // cell_width)), int(np.floor(y // cell_height))
                
                local_x = (x % cell_width)
                local_y = (y % cell_height)
                
                if apple_counter[cell_y, cell_x] < max_apples_per_cell:
                    idx = apple_counter[cell_y, cell_x]
                    transformed_coords[cell_y, cell_x, idx] = [local_x, local_y, r]
                    transformed_probs[cell_y, cell_x, idx] = [1]  
                    
                    apple_counter[cell_y, cell_x] += 1

                   
        transformed_coords = transformed_coords.tolist()
        transformed_probs = transformed_probs.tolist()

        transformed_coords_final.append(transformed_coords)
        transformed_probs_final.append(transformed_probs)

    return transformed_coords_final, transformed_probs_final

def transform_data_and_normalize(coords, probs, grid_size, max_apples_per_cell=2):
    image_height, image_width = 202, 308
    
    transformed_coords_final = []
    transformed_probs_final = []

    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]

    for i in range(len(probs)):
        transformed_coords = np.zeros((grid_size[0], grid_size[1], max_apples_per_cell, 4))
        transformed_probs = np.zeros((grid_size[0], grid_size[1], max_apples_per_cell, 1))

        apple_counter = np.zeros((grid_size[0], grid_size[1]), dtype=int)

        for j in range(len(probs[0])):
            if probs[i][j] == 1:  
                x, y, r = coords[i][j*3:j*3+3]

                if x == image_width and y == image_height:
                    cell_x = 3
                    cell_y = 3
                elif x == image_width:
                    cell_x = 3
                    cell_y = int(np.floor(y // cell_height))
                elif y == image_height:
                    cell_x = int(np.floor(x // cell_width))
                    cell_y = 3
                else:
                    cell_x, cell_y = int(np.floor(x // cell_width)), int(np.floor(y // cell_height))
                
                local_x = (x % cell_width)
                local_y = (y % cell_height)
                
                if apple_counter[cell_y, cell_x] < max_apples_per_cell:
                    idx = apple_counter[cell_y, cell_x]
                    transformed_coords[cell_y, cell_x, idx] = [local_x / cell_width, local_y / cell_height, 2*r / cell_width, 2*r / cell_height]
                    transformed_probs[cell_y, cell_x, idx] = [1]  
                    
                    apple_counter[cell_y, cell_x] += 1

                   
        transformed_coords = transformed_coords.tolist()
        transformed_probs = transformed_probs.tolist()

        transformed_coords_final.append(transformed_coords)
        transformed_probs_final.append(transformed_probs)

    return transformed_coords_final, transformed_probs_final

def reverse_transform_data(data, grid_size):
    image_height, image_width = 202, 308
    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]
    
    coords_final = []
    probs_final = []

    for batch_data in data:
        batch_coords = []
        batch_probs = []

        for y in range(grid_size[0]):
            for x in range(grid_size[1]):
                for apple_data in batch_data[y][x]:
                    local_x, local_y, r, prob = apple_data


                    global_x = x * cell_width + local_x
                    global_y = y * cell_height + local_y
                    
                    batch_coords.extend([global_x, global_y, r])
                    batch_probs.append(prob)  

        coords_final.append(batch_coords)
        probs_final.append(batch_probs)

    return coords_final, probs_final

def reverse_transform_data_and_normalization(data, grid_size):
    image_height, image_width = 202, 308
    cell_height, cell_width = image_height / grid_size[0], image_width / grid_size[1]

    coords_final = []
    probs_final = []

    for batch_data in data:
        batch_coords = []
        batch_probs = []

        for y in range(grid_size[0]):
            for x in range(grid_size[1]):
                for apple_data in batch_data[y][x]:
                    local_x, local_y, w, h , prob = apple_data


                    global_x = x * cell_width + local_x * cell_width
                    global_y = y * cell_height + local_y * cell_height
                    global_w = w * cell_width
                    global_h = h * cell_height
                    
                    batch_coords.extend([global_x, global_y, global_w, global_h])
                    batch_probs.append(prob)  

        coords_final.append(batch_coords)
        probs_final.append(batch_probs)

    return coords_final, probs_final

Functions needed to caluclate mAP values for rectangular data

In [3]:
##Function for mAP Calc
# Function for calculating Intersection over Union (IoU) between two bounding boxes
def Iou_One_Val(pred_elem, original_elem):
    # Unpack prediction coordinates
    x1_pred, y1_pred, width1_pred, height1_pred = pred_elem
    x1_pred = x1_pred - width1_pred / 2
    y1_pred = y1_pred - height1_pred / 2
    x2_pred = x1_pred + width1_pred
    y2_pred = y1_pred + height1_pred

    # Unpack original ground truth coordinates
    x1_orig, y1_orig, width1_orig, height1_orig = original_elem
    x1_orig = x1_orig - width1_orig / 2
    y1_orig = y1_orig - height1_orig / 2
    x2_orig = x1_orig + width1_orig
    y2_orig = y1_orig + height1_orig

    # Calculate the intersection coordinates
    x1_intersection = max(x1_pred, x1_orig)
    y1_intersection = max(y1_pred, y1_orig)
    x2_intersection = min(x2_pred, x2_orig)
    y2_intersection = min(y2_pred, y2_orig)

    # Check if there is no intersection
    if x1_orig == 0 and y1_orig == 0 and width1_orig == 0 and height1_orig == 0:
        return [-1]

    # Calculate the area of intersection
    if x1_intersection < x2_intersection and y1_intersection < y2_intersection:
        intersection_area = (x2_intersection - x1_intersection) * (y2_intersection - y1_intersection)
    else:
        intersection_area = 0.0

    # Calculate the area of prediction and original rectangles
    area1_pred = width1_pred * height1_pred
    area1_orig = width1_orig * height1_orig

    # Calculate the union area
    union_area = area1_pred + area1_orig - intersection_area

    # Calculate IoU
    if union_area == 0:
        iou = 0.0
    else:
        iou = intersection_area / union_area

    return iou


# Function to calculate precision and recall at different IoU thresholds
def calculate_precision_recall_at_iou(ground_truths, predictions, iou_thresholds):
    # Initialize dictionaries to store precision and recall for different IoU thresholds
    all_precisions = {iou_thresh: [] for iou_thresh in iou_thresholds}
    all_recalls = {iou_thresh: [] for iou_thresh in iou_thresholds}

    for iou_threshold in iou_thresholds:
        tp = 0  # True Positives
        fp = 0  # False Positives
        fn = len(ground_truths)  # False Negatives
        empty_gt_count = sum(1 for gt in ground_truths if gt == [0, 0, 0])  # Count empty ground truths
        temp_ground_truths = [gt for gt in ground_truths if gt != [0, 0, 0]]  # Remove empty ground truths

        for pred in predictions:
            best_iou = -1
            best_gt = None
            for gt in temp_ground_truths:
                iou = Iou_One_Val(pred, gt)
                if iou != -1 and iou >= best_iou:
                    best_iou = iou
                    best_gt = gt

            if best_iou >= iou_threshold:
                tp += 1
                temp_ground_truths.remove(best_gt)

        fn = fn - tp - empty_gt_count
        fp = len(predictions) - tp

        # Calculate precision and recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        for iou_thresh in iou_thresholds:
            all_precisions[iou_thresh].append(precision)
            all_recalls[iou_thresh].append(recall)

    return all_precisions, all_recalls


# Function to calculate Average Precision (AP) for a given set of precision and recall values
def calculate_ap(precisions, recalls):
    # Sort recalls in ascending order
    sort_order = np.argsort(recalls)
    sorted_recalls = np.array(recalls)[sort_order]
    sorted_precisions = np.array(precisions)[sort_order]

    # Add endpoints to the recall and precision lists
    sorted_recalls = np.concatenate(([0.0], sorted_recalls, [1.0]))
    sorted_precisions = np.concatenate(([0.0], sorted_precisions, [0.0]))

    # For each recall point, keep the highest precision
    for i in range(len(sorted_precisions) - 2, -1, -1):
        sorted_precisions[i] = max(sorted_precisions[i], sorted_precisions[i + 1])

    # Calculate AP as the sum of recall differences multiplied by precision
    ap = 0.0
    for i in range(1, len(sorted_recalls)):
        ap += ((sorted_recalls[i] - sorted_recalls[i - 1]) * sorted_precisions[i])

    return ap


# Function to calculate mean Average Precision (mAP) for a dataset
def mean_average_precision(dataset, iou_thresholds=np.arange(0.5, 1.0, 0.05)):
    map_values = []

    for ground_truths, predictions in dataset:
        # Calculate precision and recall at different IoU thresholds
        precisions, recalls = calculate_precision_recall_at_iou(ground_truths, predictions, iou_thresholds)
        ap_values = []

        # Calculate AP for each IoU threshold
        for iou_threshold in iou_thresholds:
            ap = calculate_ap(precisions[iou_threshold], recalls[iou_threshold])
            ap_values.append(ap)

        # Compute the mean AP across IoU thresholds
        map_values.append(np.mean(ap_values))

    # Calculate the mean mAP for the entire dataset
    mean_map = np.mean(map_values)
    return mean_map

Load data and apply preprocessing techniques

In [4]:
# Define path for each dataset
path_img_test = r'datasets_splited\dataset_3\test_aug\images'
path_csv_test = r'datasets_splited\dataset_3\test_aug\annotations'

path_img_train = r'datasets_splited\dataset_3\train_aug\images'
path_csv_train = r'datasets_splited\dataset_3\train_aug\annotations'

path_img_val = r'datasets_splited\dataset_3\val_aug\images'
path_csv_val = r'datasets_splited\dataset_3\val_aug\annotations'

# Define grid size
grid_size = (7,10)

#Functions invocation
df_circle_test = create_df(path_csv_test)
list_of_coords_test = get_bbox_cords(df_circle_test)
images_test = get_image_data(path_img_test)
list_of_prob_test = calculate_prob(list_of_coords_test)

df_circle_train = create_df(path_csv_train)
list_of_coords_train = get_bbox_cords(df_circle_train)
images_train = get_image_data(path_img_train)
list_of_prob_train = calculate_prob(list_of_coords_train)

df_circle_val = create_df(path_csv_val)
list_of_coords_val = get_bbox_cords(df_circle_val)
images_val = get_image_data(path_img_val)
list_of_prob_val = calculate_prob(list_of_coords_val)

# Split data into cells and normalize coords
transformed_coords_test, transformed_probs_test = transform_data_and_normalize(list_of_coords_test, list_of_prob_test, grid_size,max_apples_per_cell=1)
transformed_coords_train, transformed_probs_train = transform_data_and_normalize(list_of_coords_train, list_of_prob_train, grid_size,max_apples_per_cell=1)
transformed_coords_val, transformed_probs_val = transform_data_and_normalize(list_of_coords_val, list_of_prob_val, grid_size,max_apples_per_cell=1)

# Normalize images
for i in range(len(images_test)):
    images_test[i] = images_test[i] / 255

for i in range(len(images_train)):
    images_train[i] = images_train[i] / 255

for i in range(len(images_val)):
    images_val[i] = images_val[i] / 255

#Inserting data into numpy arrays and splitting to train and test set
test_set_coords = np.array(transformed_coords_test)
test_set_img = np.array(images_test)
test_set_prob = transformed_probs_test

train_set_coords = np.array(transformed_coords_train)
train_set_img = np.array(images_train)
train_set_prob = transformed_probs_train

valid_set_coords = np.array(transformed_coords_val)
valid_set_img = np.array(images_val)
valid_set_prob = transformed_probs_val

# Memory release
del df_circle_test
del list_of_coords_test
del images_test 
del list_of_prob_test 

del df_circle_train 
del list_of_coords_train 
del images_train 
del list_of_prob_train 

del df_circle_val 
del list_of_coords_val 
del images_val  
del list_of_prob_val 

del transformed_coords_test 
del transformed_probs_test 
del transformed_coords_train 
del transformed_probs_train 
del transformed_coords_val 
del transformed_probs_val 

In [78]:
# Optional: make dataset smaller
test_set_coords = test_set_coords[:40]
test_set_img = test_set_img[:40]

train_set_coords = train_set_coords[:300]
train_set_img = train_set_img[:300]

valid_set_coords = valid_set_coords[:40]
valid_set_img = valid_set_img[:40]

In [5]:
print(len(test_set_coords))
print(len(test_set_img))
print(len(train_set_coords))
print(len(train_set_img))
print(len(valid_set_coords))
print(len(valid_set_img))

896
896
7168
7168
896
896


In [6]:
# Load model weights
model = load_model(
    r'<your_file_path>',
    compile=False)

In [8]:
# Define the grid size
grid_size = (7, 10)

# Initialize an empty dataset
dataset = []

# Define sets
subset_img  = train_set_img
subset_coords = train_set_coords

# Iterate over each item in 'train_set_coords'
for i in range(len(subset_coords)):
    # Reshape the input image
    input_image_reshaped = np.expand_dims(subset_img[i], axis=0)
    
    # Make predictions using the model
    predictions = model.predict(input_image_reshaped, verbose=0)
    
    # Concatenate predictions along axis 4
    predictions = np.concatenate((predictions[0], predictions[1]), axis=4)
    
    # Reverse transform and normalize the predictions using grid_size
    dt, probab = reverse_transform_data_and_normalization(predictions, grid_size)
    
    # Reverse transform the deep nested coordinates and normalize them
    test_set_coords_tr = reverse_transform_deep_nested_coords_and_normalization([subset_coords[i]], grid_size, max_box_per_apple=1)
    
    # Apply a threshold and split the predictions
    prediction = apply_threshold_and_split_pred(dt, 0)
    
    # Flatten the prediction list
    before_nms = []
    for list_ in prediction:
        for elem in list_:
            before_nms.append(elem)
    
    # Apply Non-Maximum Suppression (NMS)
    pred_after_nms = NMS(before_nms, 0.7, 0.5, probab[0])
    
    # Extract predictions without the last element (assuming it's a class label)
    only_pred_after_nms = []
    for list_ in pred_after_nms:
        only_pred_after_nms.append(list_[:-1])
    
    # Split original coordinates
    original_coords = split_origin(test_set_coords_tr)
    
    # Filter out coordinates with a last element less than or equal to zero
    original_coords_without_zeros = []
    for elem in original_coords:
        if elem[-1] > 0:
            original_coords_without_zeros.append(elem)
    
    # Append the tuple of original coordinates and post-NMS predictions to the dataset
    dataset.append((original_coords_without_zeros, only_pred_after_nms))

# Calculate the mean average precision (mAP) for the dataset
mean_map = mean_average_precision(dataset, iou_thresholds=np.arange(0.5, 1.0, 0.05))


mean_map = mean_average_precision(dataset, iou_thresholds=np.arange(0.5, 1.0, 0.05))
print("Zbiór treningowy")
print(f"Średnia mAP: {mean_map*100} %")

Zbiór treningowy
Średnia mAP: 57.14939691834766 %


In [9]:
grid_size = (7,10)
dataset = []
subset_img  = valid_set_img
subset_coords = valid_set_coords

for i in range(len(subset_coords)):
    input_image_reshaped = np.expand_dims(subset_img[i], axis=0)
    predictions=model.predict(input_image_reshaped, verbose = 0)
    predictions = np.concatenate((predictions[0], predictions[1]), axis = 4)
    dt, probab = reverse_transform_data_and_normalization(predictions, grid_size)
    test_set_coords_tr = reverse_transform_deep_nested_coords_and_normalization([subset_coords[i]], grid_size, max_box_per_apple=1)
    prediction = apply_threshold_and_split_pred(dt, 0)
    before_nms = []
    for list_ in prediction:
        for elem in list_:
            before_nms.append(elem)
    pred_after_nms = NMS(before_nms,0.7,0.5,probab[0])
    only_pred_after_nms = []
    for list_ in pred_after_nms:
        only_pred_after_nms.append(list_[:-1])
    original_coords = split_origin(test_set_coords_tr)
    original_coords_without_zeros = []
    for elem in original_coords:
        if elem[-1] > 0:
            original_coords_without_zeros.append(elem)
    dataset.append((original_coords_without_zeros,only_pred_after_nms))

mean_map = mean_average_precision(dataset, iou_thresholds=np.arange(0.5, 1.0, 0.05))
print("Zbiór walidacyjny")
print(f"Średnia mAP: {mean_map*100} %")

Zbiór walidacyjny
Średnia mAP: 1.3803186199280637 %


In [10]:
grid_size = (7,10)
dataset = []
subset_img  = test_set_img
subset_coords = test_set_coords

for i in range(len(subset_coords)):
    input_image_reshaped = np.expand_dims(subset_img[i], axis=0)
    predictions=model.predict(input_image_reshaped, verbose = 0)
    predictions = np.concatenate((predictions[0], predictions[1]), axis = 4)
    dt, probab = reverse_transform_data_and_normalization(predictions, grid_size)
    test_set_coords_tr = reverse_transform_deep_nested_coords_and_normalization([subset_coords[i]], grid_size, max_box_per_apple=1)
    prediction = apply_threshold_and_split_pred(dt, 0)
    before_nms = []
    for list_ in prediction:
        for elem in list_:
            before_nms.append(elem)
    pred_after_nms = NMS(before_nms,0.7,0.5,probab[0])
    only_pred_after_nms = []
    for list_ in pred_after_nms:
        only_pred_after_nms.append(list_[:-1])
    original_coords = split_origin(test_set_coords_tr)
    original_coords_without_zeros = []
    for elem in original_coords:
        if elem[-1] > 0:
            original_coords_without_zeros.append(elem)
    dataset.append((original_coords_without_zeros,only_pred_after_nms))

mean_map = mean_average_precision(dataset, iou_thresholds=np.arange(0.5, 1.0, 0.05))
print("Zbiór testowy")
print(f"Średnia mAP: {mean_map*100} %")

Zbiór testowy
Średnia mAP: 2.1697364691068555 %
